In [1]:
from game_logic.board import Board, Coord
from game_logic.game import Game
from game_logic.player import Player


In [2]:
board = Board(10,10)
players = [Player(10, i) for i in range(2)]
game = Game(board=board, players=players)

In [16]:
game.action_build_unit(players[0], Coord(2,2, board), "warrior")
game.action_end_turn(players[0])
game.action_build_unit(players[1], Coord(2,3, board), "warrior")
game.action_end_turn(players[1])

ValueError: Unit already present in this cell

In [22]:
board = Board(10,10)
players = [Player(10, i) for i in range(2)]
game = Game(board=board, players=players)
game.action_build_unit(players[0], Coord(2,2, board), "warrior")
game.action_end_turn(players[0])
game.action_build_unit(players[1], Coord(2,3, board), "warrior")
game.action_end_turn(players[1])
print(game.action_attack(players[0], [0], 1))
print(game.render())

1
0: 10$
1: 10$
------------------------------
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |w0|w1|  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  


In [5]:
print(game.render())

0: 10$
1: 10$
------------------------------
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |w0|w1|  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  


In [6]:
game.units[1].stats

0: 10$
1: 10$
------------------------------
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |w0|  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  


In [8]:
game.action_attack(players[0], [0], 1)

ValueError: Unit not found with id 1

In [8]:
game.units[1].id

IndexError: list index out of range

In [ ]:
game.action_move_unit(players[0], 0, Coord(game.units[0].location.row, game.units[0].location.col+1, board))
print(game.render())

ValueError: Moving to used cell

In [9]:
print(game.render())

0: 10$
1: 10$
------------------------------
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |w0|  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
  |  |  |  |  |  |  |  |  |  
